# Widgets

In [1]:
from fastai.gen_doc.nbdoc import *
from fastai import *
from fastai.vision import *
from fastai.widgets import DatasetFormatter, ImageDeleter, ImageRelabeler

fastai offers several widgets to support the workflow of a deep learning practitioner. The purpose of the widgets are to help you organize, clean, and prepare your data for your model. Widgets are separated by data type.

## Images

### DatasetFormatter
The [`DatasetFormatter`](/widgets.image_cleaner.html#DatasetFormatter) class prepares your image dataset for widgets by returning a formatted [`DatasetTfm`](/vision.data.html#DatasetTfm) based on the [`DatasetType`](/basic_data.html#DatasetType) specified. Use `from_toplosses` to grab the most problematic images directly from your learner. Optionally, you can restrict the formatted dataset returned to `n_imgs`.

Specify the [`DatasetType`](/basic_data.html#DatasetType) you'd like to process:
- DatasetType.Train
- DatasetType.Valid
- DatasetType.Test

In [24]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [25]:
learner = create_cnn(data, models.resnet34, metrics=[accuracy])
ds, idxs = DatasetFormatter().from_toplosses(learner, ds_type=DatasetType.Valid)

NameError: name 'num_features_model' is not defined

### ImageDeleter

[`ImageDeleter`](/widgets.image_cleaner.html#ImageDeleter) is for cleaning up images that don't belong in your dataset. It renders images in a row and gives you the opportunity to delete the file from your file system.

In [4]:
ImageDeleter(ds, idxs)

NameError: name 'ds' is not defined

### ImageRelabeler

[`ImageRelabeler`](/widgets.image_cleaner.html#ImageRelabeler) helps you find mis-categorized images in your data directory.

> NOTE: ImageRelabeler currently only works with files where labels were created from the names of their parent directory (i.e. with .from_folder()). The widget moves mislabeled photos from the incorrect parent directory to the properly-labeled parent directory.

To relabel an image, just click the proper label in the widget dropdown.

In [20]:
ImageRelabeler(ds, idxs)

NameError: name 'ds' is not defined